In [1]:
import pandas as pd
from pandas_datareader import data, wb
from datetime import datetime
import numpy as np
import graphviz
import sys
import os

In [2]:
if not os.path.exists('GOOG.csv'):
    df = data.DataReader('GOOG', 'yahoo', datetime(2010, 1, 1), datetime(2017, 11, 1))
    df.to_csv('GOOG.csv')
else:
    df = pd.read_csv('GOOG.csv')

In [3]:
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,311.449310,312.721039,310.103088,311.349976,311.349976,3937800
1,2010-01-05,311.563568,311.891449,308.761810,309.978882,309.978882,6048500
2,2010-01-06,310.907837,310.907837,301.220856,302.164703,302.164703,8009000
3,2010-01-07,302.731018,303.029083,294.410156,295.130463,295.130463,12912000
4,2010-01-08,294.087250,299.675903,292.651581,299.064880,299.064880,9509900
5,2010-01-11,300.276978,300.276978,295.100647,298.612823,298.612823,14519600
6,2010-01-12,296.893982,297.147339,292.100159,293.332153,293.332153,9769600
7,2010-01-13,286.382355,292.288940,285.095734,291.648102,291.648102,13077600
8,2010-01-14,290.063416,295.180145,289.521942,293.019196,293.019196,8535300
9,2010-01-15,294.752930,294.862213,287.152344,288.126007,288.126007,10939600


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1973 entries, 0 to 1972
Data columns (total 7 columns):
Date         1973 non-null object
Open         1973 non-null float64
High         1973 non-null float64
Low          1973 non-null float64
Close        1973 non-null float64
Adj Close    1973 non-null float64
Volume       1973 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 108.0+ KB


In [5]:
df.drop('Date', axis=1, inplace=True, errors='ignore')
df = df.diff()
df.iloc[0, :] = 0
df.head()

,Open,High,Low,Close,Adj Close,Volume
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.114258,-0.829590,-1.341278,-1.371094,-1.371094,2110700.0
2,-0.655731,-0.983612,-7.540954,-7.814179,-7.814179,1960500.0
3,-8.176819,-7.878754,-6.810700,-7.034240,-7.034240,4903000.0
4,-8.643768,-3.353180,-1.758575,3.934417,3.934417,-3402100.0


In [6]:
df = df[['Open', 'High', 'Low', 'Close']]

In [7]:
df.describe()

,Open,High,Low,Close
count,1973.000000,1973.000000,1973.000000,1973.000000
mean,0.357709,0.363380,0.358260,0.361961
std,7.684202,7.012193,7.073283,7.497264
min,-66.780029,-47.500000,-47.280029,-40.369995
25%,-3.039978,-2.434173,-2.569947,-2.911042
50%,0.293091,0.239990,0.499695,0.168914
75%,3.859986,3.020020,3.530030,3.669983
max,83.880005,93.788025,80.000000,93.080017


In [8]:
n_train = int(2/3*df.shape[0])
Xtr, Xte = df.iloc[:n_train, :], df.iloc[n_train:,:]
N_FEATS = Xtr.shape[1]
N_FEATS

4

In [9]:
WND_DAYS = 14
MAX_NORM = 64

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

## Just random LSTM architecture

In [11]:
class RNN1(nn.Module):
    
    
    def __init__(self, n_in, n_mem, n_out):
        super(RNN1, self).__init__()
        self.n_in, self.n_mem, self.n_out = n_in, n_mem, n_out
        self.forget = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.learn_new = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.save_new = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.output = nn.Linear(in_features=(n_in + n_mem), out_features=n_out)
        
    def init_weights(self, var=.1):
        self.forget.weight.data.uniform_(-var, var)
        self.learn_new.weight.data.uniform_(-var, var)
        self.save_new.weight.data.uniform_(-var, var)
        self.output.weight.data.uniform_(-var, var)
    
    def forward(self, x, mem):
        # print(x.data.shape, mem.data.shape)
        xmem = torch.cat((x, mem))
        forget = F.sigmoid(self.forget(xmem))
        mem = mem*forget
        new_mem = F.tanh(self.learn_new(xmem))
        mem_mask = F.sigmoid(self.save_new(xmem))
        mem = mem + new_mem*mem_mask
        xmem = torch.cat((x, mem))
        out = self.output(xmem)
        return (out, mem)
    
    def init_mem(self):
        return Variable(torch.zeros(self.n_mem))

In [12]:
def loss(
    model,
    X,
    criterion,
    wnd=WND_DAYS,
    var='Close',
    lr=None,
    ):
    mem = model.init_mem()
    total_L = 0
    for i in range(wnd, X.shape[0]):
        model.zero_grad()
        x = X.iloc[i-wnd:i,:].as_matrix().reshape(-1)
        x = torch.Tensor(x)
        x = Variable(x)
        y = torch.Tensor(X.iloc[i:i+wnd,:].loc[:, var].as_matrix().reshape(-1))
        y = Variable(y)
        y_pred, mem = model(x, mem)
        y_pred = y_pred[:y.data.shape[0]]
        L = criterion(y, y_pred)
        total_L += L.data
        
        if lr is not None:
            L.backward(retain_graph=1)
            torch.nn.utils.clip_grad_norm(model.parameters(), MAX_NORM)
            for p in model.parameters():
                p.data.sub_(lr * p.grad.data)
    return total_L/X.shape[0]

In [13]:
rnn1 = RNN1(n_in=WND_DAYS*N_FEATS, n_mem=48, n_out=WND_DAYS)

**We'll use custom loss: a combination of a fee for wrong signs and the L2-norm of an error**

In [14]:
def objective(y_pred, y, alpha=.35):
    n = sum(y.data.shape)
    
    sgn = y_pred*y
    fee = y_pred - y
    fee[sgn > 0] = 0
    fee[fee < 0] *= -1
    # NB: you're not dividing by n yet
    fee = fee.sum()
    
    mse = (y_pred - y)**2
    mse = mse.sum()/n # TODO: don't divide by $n$
    if np.random.randint(1, 5000) == 42:
        print(dict(fee=fee, mse=mse))
        print('y:', y[sgn < 0])
        print('y_pred:', y_pred[sgn < 0])
    return (1-alpha)*fee + alpha*mse
criterion = objective

In [15]:
for epoch in range(1, 4):
    print('Epoch #%s; Loss: %s' % (epoch, loss(rnn1, Xtr, criterion, lr=.02/epoch)))

Epoch #1; Loss: 
 88.3637
[torch.FloatTensor of size 1]

{'fee': Variable containing:
 32.5060
[torch.FloatTensor of size 1]
, 'mse': Variable containing:
 50.9839
[torch.FloatTensor of size 1]
}
y: Variable containing:
 7.2489
-1.5584
-1.1587
-2.2778
-7.6393
[torch.FloatTensor of size 5]

y_pred: Variable containing:
-0.2682
 1.4655
 3.3383
 1.8778
 5.6731
[torch.FloatTensor of size 5]

Epoch #2; Loss: 
 57.2613
[torch.FloatTensor of size 1]

{'fee': Variable containing:
 29.0185
[torch.FloatTensor of size 1]
, 'mse': Variable containing:
 24.2409
[torch.FloatTensor of size 1]
}
y: Variable containing:
 0.7981
-0.0213
-2.3478
 3.3764
-1.5900
-0.9197
-0.5564
[torch.FloatTensor of size 7]

y_pred: Variable containing:
-0.0844
 6.4630
 0.1938
-4.0785
 4.2921
 3.7108
 0.5862
[torch.FloatTensor of size 7]

Epoch #3; Loss: 
 45.5634
[torch.FloatTensor of size 1]



In [17]:
rnn1

RNN1 (
  (forget): Linear (104 -> 48)
  (learn_new): Linear (104 -> 48)
  (save_new): Linear (104 -> 48)
  (output): Linear (104 -> 14)
)

In [18]:
torch.save(rnn1.state_dict(), 'rnn1.weights')

In [19]:
def next_day_predictions(model, X, wnd=WND_DAYS):
    mem = model.init_mem()
    pred = np.zeros(X.shape[0])
    for i in range(wnd, X.shape[0]):
        x = X.iloc[i-wnd:i, :].as_matrix().reshape(-1)
        x = Variable(torch.Tensor(x))
        x, mem = model(x, mem)
        pred[i] = x.data[0]
    return pred

In [20]:
import bokeh.models
import bokeh.plotting as bk
bk.output_notebook()

Loading BokehJS ...

In [21]:
def evaluate_model(model, X, title='Model evaluation'):
    X_pred = next_day_predictions(model, X)
    p = bk.figure(
        plot_width=800, plot_height=600,
        title=title,
        active_scroll='wheel_zoom')
    truth = X.Close.cumsum()
    pred = truth.shift(-1) + X_pred
    pred = pred.shift(1)
    p.line(X.index, truth, line_color='red', legend='Ground truth')
    p.line(X.index, pred, line_color='gray', legend='Prediction')
    bk.show(p)

In [22]:
evaluate_model(rnn1, Xtr, 'Model evaluation on the train set')

In [23]:
evaluate_model(rnn1, Xte, 'Model evaluation on the test set')

## Another LSTM architecture

In [24]:
class RNN2(nn.Module):
    
    
    def __init__(self, n_in, n_mem, n_out):
        super(RNN2, self).__init__()
        self.n_in, self.n_mem, self.n_out = n_in, n_mem, n_out
        self.forget = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.learn_new = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.save_new = nn.Linear(in_features=(n_in + n_mem), out_features=n_mem)
        self.x2o = nn.Linear(in_features=n_in, out_features=n_out)
        self.mem2o = nn.Linear(in_features=n_mem, out_features=n_out)
        
    def init_weights(self, var=.1):
        self.forget.weight.data.uniform_(-var, var)
        self.learn_new.weight.data.uniform_(-var, var)
        self.save_new.weight.data.uniform_(-var, var)
        self.output.weight.data.uniform_(-var, var)
    
    def forward(self, x, mem):
        # print(x.data.shape, mem.data.shape)
        xmem = torch.cat((x, mem))
        forget = F.sigmoid(self.forget(xmem))
        mem = mem*forget
        new_mem = F.tanh(self.learn_new(xmem))
        mem_mask = F.sigmoid(self.save_new(xmem))
        mem = mem + new_mem*mem_mask
        o1 = self.x2o(x)
        o2 = F.tanh(self.mem2o(mem))
        out = o1*o2
        return (out, mem)
    
    def init_mem(self):
        return Variable(torch.zeros(self.n_mem))

In [25]:
rnn2 = RNN2(n_in=WND_DAYS*N_FEATS, n_mem=48, n_out=WND_DAYS)

In [26]:
for epoch in range(1, 5):
    print('Epoch #%s; Loss: %s;' % (epoch, loss(rnn2, Xtr, criterion, lr=.02/epoch)))

Epoch #1; Loss: 
 58.6139
[torch.FloatTensor of size 1]
;
Epoch #2; Loss: 
 47.0360
[torch.FloatTensor of size 1]
;
Epoch #3; Loss: 
 38.5988
[torch.FloatTensor of size 1]
;
{'fee': Variable containing:
 16.6008
[torch.FloatTensor of size 1]
, 'mse': Variable containing:
 20.8732
[torch.FloatTensor of size 1]
}
y: Variable containing:
-0.0610
-3.2448
-0.0476
 0.7503
-1.1274
-0.7281
[torch.FloatTensor of size 6]

y_pred: Variable containing:
 2.6554
 1.7604
 1.8000
-0.7061
 1.4520
 2.2676
[torch.FloatTensor of size 6]

Epoch #4; Loss: 
 33.2716
[torch.FloatTensor of size 1]
;


In [27]:
evaluate_model(rnn2, Xtr, 'Evaluation of second model on the training set')

In [28]:
evaluate_model(rnn2, Xte, 'Evaluation of second model on the test set')

**Gosh, this one looks promising! Shall I start trading?**